# Formas de romper IF
En general la mayoria de errores que podemos cometer con IF lanzan errores en tiempo de ejecucion (por ejemplo escribir IF en mayusculas), o usar llaves cuadradas cuando no se necesita, o que nos falte un espacio, lo que las saca de nuestro alcance normal (aun asi shellheck tiene alertas de eso), tenemos un par de casos donde podemos romper facilmente cosas que funcionan bien, o hacer que algo paresca que funciona, pero en realidad no lo hace...

Tomemos como ejemplo esto:

In [2]:
cat sources/bash/badif1.sh
bash -x sources/bash/badif1.sh

#!/bin/env bash
if [ 1 -eq 1 ]
then
  pwd
fi
+ '[' 1 -eq 1 ']'
+ pwd
/home/demo


pero resulta que el pwd solo era un debug y hemos desido comentarlo

In [6]:
cat sources/bash/badif2.sh
bash -x sources/bash/badif2.sh
shellcheck sources/bash/badif2.sh

#!/bin/env bash
if [ 1 -eq 1 ]
then
  #pwd
fi
sources/bash/badif2.sh: line 5: syntax error near unexpected token `fi'
sources/bash/badif2.sh: line 5: `fi'

In sources/bash/badif2.sh line 2:
if [ 1 -eq 1 ]
^-- SC1009: The mentioned parser error was in this if expression.


In sources/bash/badif2.sh line 3:
then
^-- SC1073: Couldn't parse this then clause.


In sources/bash/badif2.sh line 5:
fi
^-- SC1048: Can't have empty then clauses (use 'true' as a no-op).
  ^-- SC1072: Unexpected keyword/token. Fix any mentioned problems and try again.



: 1

no podemos tener ramas en los if que sean vacias (esto es cierto en muchos lenguajes de programacion) y en bash los comentarios no cuentan como codigo, shellchek recomienda usar "true" pero puedes usar ":" como comando abierto

In [8]:
cat sources/bash/badif3.sh
bash -x sources/bash/badif3.sh
shellcheck sources/bash/badif3.sh

#!/bin/env bash
if [ 1 -eq 1 ]
then
  #pwd
  :
fi
+ '[' 1 -eq 1 ']'
+ :

In sources/bash/badif3.sh line 2:
if [ 1 -eq 1 ]
       ^-- SC2050: This expression is constant. Did you forget the $ on a variable?



: 1

y funciona aun que tengas mas comandos

In [9]:
cat sources/bash/badif4.sh
bash -x sources/bash/badif4.sh
shellcheck sources/bash/badif4.sh

#!/bin/env bash
if [ 1 -eq 1 ]
then
  pwd
  :
fi
+ '[' 1 -eq 1 ']'
+ pwd
/home/demo
+ :

In sources/bash/badif4.sh line 2:
if [ 1 -eq 1 ]
       ^-- SC2050: This expression is constant. Did you forget the $ on a variable?



: 1